<a href="https://colab.research.google.com/github/sergak0/AIIJC/blob/main/Lisa/KeywordBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 7.7 MB/s 
     |████████████████████████████████| 371 kB 36.3 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=848618 sha256=a8f6bcd400ae1472dd566c1bf1bf39581648706682fec7732c6cb023985113dd
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [15]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from typing import List
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from typing import List
import nltk
import string
import pymorphy2
import codecs
from gensim.models import KeyedVectors

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
class Preparator():
    def __init__(self,
                 preparator_fold_name: str = '/content/drive/MyDrive/aiijc_sber/'):
        stopwords_path = preparator_fold_name + 'all_stopwords.txt'
        bigram_model_path = preparator_fold_name + 'bigram_model.pkl'
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.stopwords = set(line.strip() for line in codecs.open(stopwords_path, "r", "utf_8_sig").readlines())
        self.bigram_mod = gensim.models.Phrases.load(bigram_model_path)

    def prepare_corp(self, news_list: List[str]):
        return [self.newstext2token(news_text) for news_text in news_list]

    def newstext2token(self, news_text: str):
        tokens = self.tokenizer.tokenize(news_text.lower())
        tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if
                                all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю' for c in w)]
        tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
        tokens_last = [w for w in tokens_base_forms if len(w) > 1]
        tokens_bigrammed = self.make_bigrams(tokens_last)
        return ' '.join(tokens_bigrammed)

    def make_bigrams(self, doc):
        return self.bigram_mod[doc]

    def prepare_texts(self, texts):
        return self.prepare_corp(texts)



In [35]:
data = pd.read_csv('/content/drive/MyDrive/aiijc_sber/tokens_full_cleaned.csv')
fixed_tokens = []
empty = 0
for  tok in data.tokens:
  aslist = tok.split()
  fixed_tokens.append(aslist)
  empty += aslist.count('')
data.tokens = fixed_tokens
data = data.drop(['Unnamed: 0'], axis=1)
data.head()

,texts,y,tokens
0,Живо́тные (лат. Animalia) — традиционно (со вр...,0,"[живо, тный, лата, аристотель, выделять, орган..."
1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,0,"[пти, цы, лата, группа, теплокровный, яйцеклад..."
2,Союз охраны птиц России (СОПР) — общероссийска...,0,"[союз, охрана, птица, россия, сопра, общеросси..."
3,Крючковидные отростки рёбер — костяные выросты...,0,"[крючковидный, отросток, ребро, костяной, выро..."
4,Myxozoa (лат.) — класс или подтип паразитическ...,0,"[лата, класс, подтип, паразитический, беспозво..."


In [36]:
from gensim.models import Word2Vec
model = Word2Vec(data.tokens, window=5, min_count=2)

In [37]:
model.wv.most_similar('птица', topn=10)

[('рыба', 0.9730218648910522),
 ('черепаха', 0.9595657587051392),
 ('пчела', 0.9575172066688538),
 ('игуана', 0.9484673738479614),
 ('насекомое', 0.9480222463607788),
 ('эпиорнис', 0.9392580389976501),
 ('хищный', 0.9373494982719421),
 ('засолить', 0.9346617460250854),
 ('млекопитающее', 0.934207558631897),
 ('многощетинковый', 0.9336563944816589)]

In [14]:
model.wv.save("/content/drive/MyDrive/aiijc_sber/keyword_w2v.kv")

In [18]:
def text2vec(tokens: List[str], embeddings: KeyedVectors) -> np.ndarray:
    relevant = 0
    words_vecs = np.zeros((embeddings.vector_size,))
    for word in tokens:
        if word in embeddings:
            words_vecs += embeddings.word_vec(word, True)
            relevant += 1

    if relevant:
        words_vecs /= relevant
    return words_vecs

In [16]:
def extract_keywords(tokens: List[str],
                     embeddings: KeyedVectors,
                     top_n: int = 5,
                     diversity: float = 0.5,
                     nr_candidates: int = 20) -> str:
    candidates = []
    doc_embedding = text2vec(tokens, embeddings).reshape(1, -1)
    candidate_embeddings = []
    for candidate in tokens:
        if candidate in embeddings:
            candidates.append(candidate)
            candidate_embeddings.append(embeddings[candidate])

    if len(candidates) == 0:
        return tokens

    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]][::-1]

    return ' '.join(set(keywords))




In [173]:
data = pd.read_csv('/content/drive/MyDrive/aiijc_sber/good_additional_dataset.csv')

In [174]:
data.head()

,Unnamed: 0,texts,prediction,score
0,0,\t1.1. Летом в спортивный лагерь ходили 50 дет...,спорт,0.496684
1,1,\t1.2. На концерте в летнем лагере ребята игра...,музыка,0.621388
2,3,\t1.4. Во время летних соревнований по плавани...,спорт,0.646785
3,4,\t1.5. В спортивную школу во время летних кани...,спорт,0.640304
4,6,\t1.7. Самые крупные животные на нашей планете...,животные,0.407655


In [175]:
data['tokens'] = [i.split() for i in Preparator().prepare_corp(data.texts)]

In [176]:
data.head()

,Unnamed: 0,texts,prediction,score,tokens
0,0,\t1.1. Летом в спортивный лагерь ходили 50 дет...,спорт,0.496684,"[спортивный, лагерь, ребёнок, мальчик, спортив..."
1,1,\t1.2. На концерте в летнем лагере ребята игра...,музыка,0.621388,"[концерт, летний, ребята, гитара, гитара, стру..."
2,3,\t1.4. Во время летних соревнований по плавани...,спорт,0.646785,"[летний, соревнование, плавание, ребята, посет..."
3,4,\t1.5. В спортивную школу во время летних кани...,спорт,0.640304,"[спортивный, школа, летний, каникулы, купить, ..."
4,6,\t1.7. Самые крупные животные на нашей планете...,животные,0.407655,"[животное, синий, кит, весить, длина, кит, пос..."


In [177]:
highlights = [extract_keywords(toks, model.wv, 10) for toks in data.tokens]
data['highlights'] = highlights

In [178]:
data.head()

,Unnamed: 0,texts,prediction,score,tokens,highlights
0,0,\t1.1. Летом в спортивный лагерь ходили 50 дет...,спорт,0.496684,"[спортивный, лагерь, ребёнок, мальчик, спортив...",спортивный лагерь мальчик ребёнок
1,1,\t1.2. На концерте в летнем лагере ребята игра...,музыка,0.621388,"[концерт, летний, ребята, гитара, гитара, стру...",струна концерт гитара летний ребята
2,3,\t1.4. Во время летних соревнований по плавани...,спорт,0.646785,"[летний, соревнование, плавание, ребята, посет...",дорожка соревнование плавательный плавание про...
3,4,\t1.5. В спортивную школу во время летних кани...,спорт,0.640304,"[спортивный, школа, летний, каникулы, купить, ...",мяч каникулы летний теннисный купить волейболь...
4,6,\t1.7. Самые крупные животные на нашей планете...,животные,0.407655,"[животное, синий, кит, весить, длина, кит, пос...",высота весить голова длина кит жираф синий


In [179]:
data.to_csv('/content/drive/MyDrive/aiijc_sber/good_additional_dataset_keyworded.csv')

In [180]:
class BayesModel():

    def __init__(self,
                 pretrained=True,
                 nb_fold_name: str = '/content/drive/MyDrive/aiijc_sber/nb/'):

        if not pretrained:
            self.vect = CountVectorizer()
            self.tfidf = TfidfTransformer()
            self.clf = MultinomialNB()
        else:
            self.vect = pickle.load(open(nb_fold_name + 'nb_vect.sav', 'rb'))
            self.tfidf = pickle.load(open(nb_fold_name + 'nb_tfidf.sav', 'rb'))
            self.clf = pickle.load(open(nb_fold_name + 'nb_clf.sav', 'rb'))

        self.nb = Pipeline([('vect', self.vect),
                            ('tfidf', self.tfidf),
                            ('clf', self.clf),
                            ])

        self.classes = ['животные', "музыка", "спорт", "литература"]

    def fit(self, X_train, Y_train):
        self.nb.fit(X_train, Y_train)

    def predict(self, X_test: List[str]):
        return self.nb.predict(X_test)

    def predict_proba(self, X_test: List[str]):
        return self.nb.predict_proba(X_test)

    def show_metrics(self, X_test, Y_test):
        y_pred = self.predict(X_test)
        print('accuracy %s' % accuracy_score(y_pred, Y_test))
        print(classification_report(Y_test, y_pred, target_names=self.classes))

    def score(self, X_test, Y_test):
        return self.nb.score(X_test, Y_test)

    def save_models(self,
                    nb_fold_name: str = '/content/drive/MyDrive/aiijc_sber/nb/'):
        pickle.dump(self.vect, open(nb_fold_name + "nb_vect.sav", 'wb'))
        pickle.dump(self.tfidf, open(nb_fold_name + 'nb_tfidf.sav', 'wb'))
        pickle.dump(self.clf, open(nb_fold_name + 'nb_clf.sav', 'wb'))


In [181]:
nb_model = BayesModel(False)

In [182]:
X_train, X_test, Y_train, Y_test = train_test_split(data.highlights, data.prediction, test_size = 0.1)

In [183]:
X_train

303                                       сантиметр метр
25               мяч остаться взять счёт урок математика
290              закончиться путешествие август начаться
61                   столько остаться штука съесть мешок
354                                  спрятать дупло орех
                             ...                        
282                                      пройти остаться
589                               коза фермер овца серый
36     позвоночник копчиковый шейный поясничный тело ...
123    остаться соревнование легкоатлет готовиться ст...
732    принять соревнование спортсмен команда гимнаст...
Name: highlights, Length: 798, dtype: object

In [184]:
Y_train

303         спорт
25          спорт
290    литература
61       животные
354      животные
          ...    
282         спорт
589      животные
36       животные
123         спорт
732         спорт
Name: prediction, Length: 798, dtype: object

In [185]:
nb_model.fit(X_train, Y_train)

In [186]:
nb_model.show_metrics(X_test, Y_test)

accuracy 0.8089887640449438
              precision    recall  f1-score   support

    животные       0.71      0.97      0.82        31
      музыка       0.81      0.84      0.82        25
       спорт       1.00      0.64      0.78        22
  литература       1.00      0.64      0.78        11

    accuracy                           0.81        89
   macro avg       0.88      0.77      0.80        89
weighted avg       0.85      0.81      0.81        89



In [187]:
nb_model.save_models('/content/drive/MyDrive/aiijc_sber/nb_keys/')

In [188]:
tests = pd.read_csv('/content/drive/MyDrive/aiijc_sber/test.csv')
tests.columns = ['id', 'texts']
tests.head()

,id,texts
0,0,Вопрос 1 Денис готовится провести соревнования...
1,1,Реши задачу.\nСколько коробок корма для собак ...
2,2,"Если групп элементов несколько, то применяется..."
3,3,Вопрос1 ...
4,4,Для наглядной геометрической иллюстрации объём...


In [189]:
tests['prediction'] = nb_model.predict(Preparator().prepare_texts(tests.texts))

In [190]:
tests.head()

,id,texts,prediction
0,0,Вопрос 1 Денис готовится провести соревнования...,спорт
1,1,Реши задачу.\nСколько коробок корма для собак ...,литература
2,2,"Если групп элементов несколько, то применяется...",животные
3,3,Вопрос1 ...,животные
4,4,Для наглядной геометрической иллюстрации объём...,животные


In [191]:
for id, st in tests[:20].iterrows():
  print(st['prediction'])
  print(st.texts)
  print()

спорт
Вопрос 1 Денис готовится провести соревнования по настольному хоккею и решил распечатать недостающие фигурки на принторе . Для изготовления одной 3D-модели у Дениса слишком мало времени, поэтому он решает использовать 2 принтера сразу, а потом склеить полученные детали. У специалиста сейчас в наличии 8 принтеров. Сколько у него есть вариантов выбора, если сначала он настраивает один из принтеров, а за ним — второй? 

литература
Реши задачу.
Сколько коробок корма для собак поместится в ящике, если все стороны ящика соответственно в 12 раз больше сторон коробки корма?

животные
Если групп элементов несколько, то применяется закон сложения. Если необходимо выбрать  элементы из одной группы и элементы из другой, то количество способов выбрать элементы из обеих групп будет равно 
Исходя из этого, реши задачу.
Для получения зачета по физической культуре учащимся нужно выбрать направление в котром бы они занимались в течение года и сдавали зачет. По легкой отелитики предполагается 1 гру